# **RESO Field Usage Statistics**

<p> I was curious to extra usage trends of fields using the aggregated statistics on the RESO Dictionary website. Each RESO field is a separate page and contains a Usage section at the bottom with percentages of how much each field is used across the industry - they're separated by usage of Systems and Organizations. In order to extract this data, I'd need to aggregate all the links for the RESO field pages and then extract the percentages from the Usage section of each page. <p>

<p> The pages are dynamically generated HTML and that provides some issues when scraping the site to retrieve the list of all the page links. I'd also like to include which Group the field belongs to, current issue is that some fields do not belong to a Group and the script returns an error when attempting to extract that HTML element. The script works properly as it currenly is, however there are some improvements (included below) that I have in mind to better scrape the data. Will upload these in later versions <p>

### To Do
- Include some wait and timeouts. Script doesn't retrieve all the links every time, I think this is due to not enough time to load the HTML content successfully. 
- Add ability to retrieve the Group for each field and give an Unassigned Group name to those fields causing issues that don't have a group. 
- Update code to traverse the page tree on the side to extract all the links from all resources. Page tree is dynamically generated and will likely need a large overhaul to the code

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

driver = webdriver.Chrome(options=options)

df = pd.DataFrame(columns = ['Field', 'System Usage (%)', 'Org Usage (%)', 'Resource'])
# df = pd.DataFrame(columns = ['Field', 'System Usage (%)', 'Org Usage (%)', 'Resource', 'Group'])

# Open the URL
url = 'https://ddwiki.reso.org/display/DDW17/Property+Resource'
driver.get(url)

# identify elements with tagname <a>
links = driver.find_elements(By.TAG_NAME, "a")

all_href = []

# traverse list
for lnk in links:
   # get_attribute() to get all href
   all_href.append(lnk.get_attribute('href'))

In [3]:
split_links = [i.split('?', 1)[0] for i in all_href]
field_links = [j for j in split_links if '+Field' in j]

print(field_links)

['https://ddwiki.reso.org/display/DDW17/BusinessName+Field', 'https://ddwiki.reso.org/display/DDW17/BusinessType+Field', 'https://ddwiki.reso.org/display/DDW17/HoursDaysOfOperationDescription+Field', 'https://ddwiki.reso.org/display/DDW17/HoursDaysOfOperation+Field', 'https://ddwiki.reso.org/display/DDW17/LaborInformation+Field', 'https://ddwiki.reso.org/display/DDW17/LeaseAmount+Field', 'https://ddwiki.reso.org/display/DDW17/LeaseAmountFrequency+Field', 'https://ddwiki.reso.org/display/DDW17/LeaseAssignableYN+Field', 'https://ddwiki.reso.org/display/DDW17/LeaseExpiration+Field', 'https://ddwiki.reso.org/display/DDW17/LeaseRenewalOptionYN+Field', 'https://ddwiki.reso.org/display/DDW17/NumberOfFullTimeEmployees+Field', 'https://ddwiki.reso.org/display/DDW17/NumberOfPartTimeEmployees+Field', 'https://ddwiki.reso.org/display/DDW17/OwnershipType+Field', 'https://ddwiki.reso.org/display/DDW17/SeatingCapacity+Field', 'https://ddwiki.reso.org/display/DDW17/SpecialLicenses+Field', 'https://ddw

In [4]:
for lnk in field_links:
    page = requests.get(lnk)
    soup = BeautifulSoup(page.text, 'html')
    inner_cell_elements = soup.find_all('div', class_='innerCell')

    field_name = soup.find('title').text.strip().split(' ')[0]

    crumbs = soup.find_all('ol', id='breadcrumbs')

    for a in crumbs:
        a_tag = a.find_all('a')
        a_text = [txt.text.strip() for txt in a_tag]

    if len(a_text) < 3:
        a_text.insert(3, 'null')

    for li in inner_cell_elements[6:7]:
        li_tag = li.find_all('li')
        li_text = [data.text.strip() for data in li_tag[0:2]]

    df.loc[len(df.index)] = [field_name, li_text[0], li_text[1], a_text[2]]

print(df)

                               Field        System Usage (%)  \
0                       BusinessName  65% of Systems (11/17)   
1                       BusinessType  71% of Systems (12/17)   
2    HoursDaysOfOperationDescription   18% of Systems (3/17)   
3               HoursDaysOfOperation   41% of Systems (7/17)   
4                   LaborInformation   29% of Systems (5/17)   
..                               ...                     ...   
608                  PowerProduction   <1% of Systems (0/17)   
609              PowerProductionType   35% of Systems (6/17)   
610                            Sewer  88% of Systems (15/17)   
611                        Utilities  82% of Systems (14/17)   
612                      WaterSource  82% of Systems (14/17)   

                      Org Usage (%)           Resource  
0    30% of Organizations (151/501)  Property Resource  
1    69% of Organizations (344/501)  Property Resource  
2       2% of Organizations (9/501)  Property Resource  
3  

In [5]:
df['Resource'].replace(' Resource','', regex=True, inplace=True)
df.replace({'(?<=%).*': ''}, regex=True, inplace=True)
df = df[~df['System Usage (%)'].str.contains("[a-zA-Z]").fillna(False)]

In [7]:
df.replace({'<': ''}, regex=True, inplace=True)
df['System Usage (%)'] = df['System Usage (%)'].str.replace('%', '')
df['Org Usage (%)'] = df['Org Usage (%)'].str.replace('%', '')

df = df.astype({"System Usage (%)":"int","Org Usage (%)":"int"})

display(df.dtypes)

Field               object
System Usage (%)     int64
Org Usage (%)        int64
Resource            object
dtype: object

In [8]:
df.sort_values('Org Usage (%)', ascending=False, inplace=True)

df.head(10)

,Field,System Usage (%),Org Usage (%),Resource
357,ListPrice,100,99,Property
314,ModificationTimestamp,100,99,Property
393,StateOrProvince,100,99,Property
449,BedroomsTotal,100,99,Property
331,ListingId,100,99,Property
433,PropertyType,100,99,Property
311,ListingContractDate,94,98,Property
391,PostalCode,94,98,Property
417,Longitude,94,98,Property
416,Latitude,94,98,Property


In [ ]:
df.to_csv('../RESO Scraper/reso_field_usage.csv')